In [1]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2 MB 56 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.2 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
import numpy as np
import cv2
from scipy.misc import imresize
from moviepy.editor import VideoFileClip

from keras.models import load_model
import matplotlib.pyplot as plt


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1703936/45929032 bytes (3.7%)4169728/45929032 bytes (9.1%)6758400/45929032 bytes (14.7%)9216000/45929032 bytes (20.1%)11804672/45929032 bytes (25.7%)14458880/45929032 bytes (31.5%)17055744/45929032 bytes (37.1%)19488768/45929032 bytes (42.4%)22167552/45929032 bytes (48.3%)25010176/45929032 bytes (54.5%)27967488/45929032 bytes (60.9%)30531584/45929032 bytes (66.5%)334

In [3]:
# Load Keras model
model = load_model('/content/drive/MyDrive/Colab Notebooks/lane detection tensorflow/Saved models/lanemodelfinal.h5')


In [4]:

class Lanes():
    def __init__(self):
        self.recent_fit = []
        self.avg_fit = []

def road_lines(image):
    """ Takes in a road image, re-sizes for the model,
    predicts the lane to be drawn from the model in G color,
    recreates an RGB image of a lane and merges with the
    original road image.
    """
    image_shape=image.shape
    # Get image ready for feeding into model
    small_img = imresize(image, (80, 160, 3))
    small_img = np.array(small_img)
    small_img = small_img[None,:,:,:]

    # Make prediction with neural network (un-normalize value by multiplying by 255)
    prediction = model.predict(small_img)[0] *255 
    # Add lane prediction to list for averaging
    lanes.recent_fit.append(prediction)

    # Only using last five for average
    if len(lanes.recent_fit) > 5:
        lanes.recent_fit = lanes.recent_fit[1:]

    # Calculate average detection
    lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis = 0)

    # Generate fake R & B color dimensions, stack with G
    blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)

    #for setting different color for the detection
    lane_drawn = np.dstack(((lanes.avg_fit), (lanes.avg_fit),blanks))

    # Re-size to match the original image
    lane_image = imresize(lane_drawn, image_shape)

    #plt.imshow(lane_image)
    #plt.show()


    # Merge the lane drawing onto the original image
    result = cv2.addWeighted(image, 1, lane_image, 1, 0)

    return result

lanes = Lanes()


In [5]:
# Where to save the output video
vid_output = '/content/drive/MyDrive/Colab Notebooks/lane detection tensorflow/1.mp4'

clip1 = VideoFileClip("/content/drive/MyDrive/Colab Notebooks/lane detection tensorflow/test dataset/lanes_clip.mp4")

vid_clip = clip1.fl_image(road_lines)
vid_clip.write_videofile(vid_output, audio=False)

[MoviePy] >>>> Building video /content/drive/MyDrive/Colab Notebooks/lane detection tensorflow/1.mp4
[MoviePy] Writing video /content/drive/MyDrive/Colab Notebooks/lane detection tensorflow/1.mp4


100%|█████████▉| 1297/1298 [03:19<00:00,  6.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/Colab Notebooks/lane detection tensorflow/1.mp4 

